## Imports

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx 

import folium
import json
import geopandas as gpd
from shapely.geometry import LineString
import json
import heapq
from collections import defaultdict, deque
import queue

import random
import os 

## Data

In [2]:
df_links = pd.read_csv("IC_NETWORK/df_links.csv") 
df_distances = pd.read_csv("distance_matrix.csv") 

In [3]:
df_links.start_node.unique()

array(['Zürich HB', 'Winterthur', 'Basel SBB', 'Fribourg/Freiburg',
       'Thun', 'Lugano Nord', 'Olten', 'Biel/Bienne', 'Lausanne', 'Zug',
       'Luzern', 'Genève'], dtype=object)

In [23]:
df_links.head()

,start_node,end_node,distance [m],IC_lines
0,Zürich HB,Chur,116075.402,IC3
1,Zürich HB,Winterthur,26787.367,"IC5, IC81"
2,Zürich HB,Baden,22587.175,IC3
3,Zürich HB,Bern,120492.589,"IC1, IC8, IC81"
4,Zürich HB,Zürich Oerlikon,5368.417,IC1


In [24]:
df_distances.head()

,Unnamed: 0,Bern,Basel SBB,Lausanne,Luzern,St. Gallen,Winterthur,Zug,Aarau,Baden,...,Genève-Aéroport,Genève,Neuchâtel,Olten,Thun,Bellinzona,Lugano,Lugano Nord,Zürich Oerlikon,Zürich HB
0,Bern,0.000,104798.677,96888.245,119979.281,202576.446,145437.404,141853.610,78999.684,110068.246,...,163033.063,157114.875,62661.789,65626.280,31264.012,285352.211,314907.207,314247.273,124018.454,120492.589
1,Basel SBB,104798.677,0.000,177474.735,94940.490,176122.563,118983.521,116814.819,52545.801,83614.363,...,233259.129,227340.941,102386.778,39172.397,132299.575,260313.420,289868.416,289208.482,97564.571,94038.706
2,Lausanne,96888.245,177474.735,0.000,216867.526,299464.691,242325.649,238741.855,175887.929,206956.491,...,66144.818,60226.630,75087.957,162514.525,128152.257,382240.456,411795.452,411135.518,220906.699,217380.834
3,Luzern,119979.281,94940.490,216867.526,0.000,139430.850,82291.808,28417.295,69141.497,78091.616,...,275911.172,269992.984,145038.821,55768.093,147480.179,169866.354,199421.350,198761.416,60872.858,57346.993
4,St. Gallen,202576.446,176122.563,299464.691,139430.850,0.000,57139.042,111013.555,123576.762,104671.032,...,357093.245,351175.057,226220.894,136950.166,230077.344,304803.780,334358.776,333698.842,78557.992,83926.409


### Data manipulation

To correctly model the network with transfers, we decided to model the transfers between lines at a station as seperate edges. By doing this we can have full control over transfer costs, line headways, ...

In [ ]:
# Function to split multiple train lines and create individual connections
def process_multi_line_connections(df_links):

    """Expand rows with multiple train lines into individual connections"""
    
    rows = []
    
    for _, row in df_links.iterrows():
        # Check if there are multiple lines (comma-separated)
        if ',' in str(row['IC_lines']):
            # Split the lines and create a row for each line
            for line in [l.strip() for l in row['IC_lines'].split(',')]:
                new_row = row.copy()
                new_row['IC_lines'] = line
                rows.append(new_row)
        else:
            # Single line connection
            rows.append(row)
    
    return pd.DataFrame(rows)

In [ ]:
# Create bidirectional network with individual train lines
def create_bidirectional_network(df_links):
    # First, process multi-line connections
    df_expanded = process_multi_line_connections(df_links)
    
    # Create reversed connections for bidirectional network
    df_reversed = df_expanded.copy()
    df_reversed['start_node'], df_reversed['end_node'] = df_reversed['end_node'], df_reversed['start_node']
    
    # Combine original and reversed connections
    df_bidirectional = pd.concat([df_expanded, df_reversed], ignore_index=True)
    
    return df_bidirectional

In [ ]:
# Add transfer edges between lines at the same station
def add_transfer_edges(df_bidirectional, transfer_time=5):
    
    """Add edges representing transfers between different lines at the same station"""
    
    # Get all unique station-line combinations
    station_lines = []
    for _, row in df_bidirectional.iterrows():
        station_lines.append((row['start_node'], row['IC_lines']))
    
    # Remove duplicates
    station_lines = list(set(station_lines))
    
    # Find stations with multiple lines
    stations = {}
    for station, line in station_lines:
        if station not in stations:
            stations[station] = []
        stations[station].append(line)
    
    # Create transfer edges
    transfer_edges = []
    
    for station, lines in stations.items():
        if len(lines) > 1:  # Only stations with multiple lines need transfers
            for i, line1 in enumerate(lines):
                for line2 in lines[i+1:]:
                    # Create bidirectional transfer edges
                    transfer_edges.append({
                        'start_node': station,
                        'end_node': station,
                        'distance [m]': 0,
                        'IC_lines': f"TRANSFER_{line1}_to_{line2}",
                        'transfer_time': transfer_time,
                        'is_transfer': True
                    })
                    transfer_edges.append({
                        'start_node': station,
                        'end_node': station,
                        'distance [m]': 0,
                        'IC_lines': f"TRANSFER_{line2}_to_{line1}",
                        'transfer_time': transfer_time,
                        'is_transfer': True
                    })
    
    # Convert to DataFrame
    transfer_df = pd.DataFrame(transfer_edges)
    
    # Add is_transfer column to original DataFrame
    df_bidirectional['is_transfer'] = False
    
    # Combine regular and transfer edges
    full_network = pd.concat([df_bidirectional, transfer_df], ignore_index=True)

    return full_network
    

In [ ]:
# Calculate travel times on edges
def calculate_travel_times(network_df, speed_kmh=80):
    
    """Convert distances to travel times and add transfer penalties"""
    
    network_df['travel_time'] = np.where(
        network_df['is_transfer'],
        network_df['transfer_time'],
        network_df['distance [m]'] / 1000 / speed_kmh * 60  # Convert to minutes
    )
    
    return network_df

In [ ]:
# Create node-line pairs for modified Dijkstra
def create_node_line_pairs(network_df):
    """Create unique identifiers for each (station, line) combination"""
    
    # For regular edges: add source line and destination line
    network_df['source_line'] = network_df['IC_lines']
    
    # For transfer edges: extract source and destination lines
    transfer_mask = network_df['is_transfer']
    network_df.loc[transfer_mask, 'source_line'] = network_df.loc[transfer_mask, 'IC_lines'].apply(
        lambda x: x.split('_')[1]
    )
    network_df.loc[transfer_mask, 'dest_line'] = network_df.loc[transfer_mask, 'IC_lines'].apply(
        lambda x: x.split('_')[3]
    )
    
    # For regular edges: destination line is the same as source line
    network_df.loc[~transfer_mask, 'dest_line'] = network_df.loc[~transfer_mask, 'source_line']
    
    # Create node-line identifiers
    network_df['source_id'] = network_df['start_node'] + "_" + network_df['source_line']
    network_df['dest_id'] = network_df['end_node'] + "_" + network_df['dest_line']
    
    return network_df

In [6]:
df_reversed = df_links.rename(columns={
    'start_node': 'end_node',
    'end_node': 'start_node'
})

df_links_bidirectional = pd.concat([df_links, df_reversed], ignore_index=True)

df_links_bidirectional = df_links_bidirectional.sort_values(['start_node', 'end_node']).reset_index(drop=True)

df_link = df_links_bidirectional.copy()
df_link.head()

,start_node,end_node,distance [m],IC_lines
0,Aarau,Olten,13373.404,IC5
1,Aarau,Zürich HB,41492.905,IC5
2,Baden,Basel SBB,83614.363,IC3
3,Baden,Zürich HB,22587.175,IC3
4,Basel SBB,Baden,83614.363,IC3


In [7]:
# Assuming df_links_bidirectional has 'start_node' (string) and 'link_index'

df_links_bidirectional = df_links_bidirectional.sort_values(['start_node', 'end_node']).reset_index(drop=True)
df_links_bidirectional['link_index'] = df_links_bidirectional.index

# Group by start_node and get min and max link_index per node
grouped = df_links_bidirectional.groupby('start_node')['link_index'].agg(['min', 'max']).reset_index()

# Create dicts for first and last outflow link indices per node
frst_out = dict(zip(grouped['start_node'], grouped['min']))
lst_out = dict(zip(grouped['start_node'], grouped['max']))

# For nodes without outgoing links, you can fill with -1 if needed:
all_nodes = set(df_links_bidirectional['start_node']).union(df_links_bidirectional['end_node'])
for node in all_nodes:
    if node not in frst_out:
        frst_out[node] = -1
    if node not in lst_out:
        lst_out[node] = -1

print(frst_out)
print(lst_out)

df_node = pd.DataFrame({
    'node_id': list(frst_out.keys()),
    'frst_out': list(frst_out.values()),
    'lst_out': list(lst_out.values())
})

df_node = df_node.sort_values('node_id').reset_index(drop=True)

df_node.head()

{'Aarau': 0, 'Baden': 2, 'Basel SBB': 4, 'Bellinzona': 7, 'Bern': 10, 'Biel/Bienne': 14, 'Chur': 17, 'Fribourg/Freiburg': 18, 'Genève': 20, 'Genève-Aéroport': 22, 'Lausanne': 23, 'Lugano': 26, 'Lugano Nord': 27, 'Luzern': 29, 'Neuchâtel': 31, 'Olten': 33, 'St. Gallen': 38, 'Thun': 39, 'Winterthur': 40, 'Zug': 43, 'Zürich HB': 45, 'Zürich Oerlikon': 52}
{'Aarau': 1, 'Baden': 3, 'Basel SBB': 6, 'Bellinzona': 9, 'Bern': 13, 'Biel/Bienne': 16, 'Chur': 17, 'Fribourg/Freiburg': 19, 'Genève': 21, 'Genève-Aéroport': 22, 'Lausanne': 25, 'Lugano': 26, 'Lugano Nord': 28, 'Luzern': 30, 'Neuchâtel': 32, 'Olten': 37, 'St. Gallen': 38, 'Thun': 39, 'Winterthur': 42, 'Zug': 44, 'Zürich HB': 51, 'Zürich Oerlikon': 53}


,node_id,frst_out,lst_out
0,Aarau,0,1
1,Baden,2,3
2,Basel SBB,4,6
3,Bellinzona,7,9
4,Bern,10,13


## Create a random OD table to start working 

In [8]:
cities = ['Genève-Aéroport', 'Genève', 'Zürich Oerlikon', 'Zürich HB',
          'Thun', 'Basel SBB', 'Lausanne', 'Winterthur', 'St. Gallen',
          'Luzern', 'Lugano', 'Baden', 'Fribourg/Freiburg', 'Biel/Bienne',
          'Neuchâtel', 'Bellinzona', 'Zug', 'Bern', 'Olten', 'Chur', 'Aarau']

n = len(cities)

np.random.seed(42)  # for reproducibility
od_matrix = np.random.randint(0, 10000, size=(n, n))

np.fill_diagonal(od_matrix, 0)

df_od = pd.DataFrame(od_matrix, index=cities, columns=cities)

df_od.head()

,Genève-Aéroport,Genève,Zürich Oerlikon,Zürich HB,Thun,Basel SBB,Lausanne,Winterthur,St. Gallen,Luzern,...,Baden,Fribourg/Freiburg,Biel/Bienne,Neuchâtel,Bellinzona,Zug,Bern,Olten,Chur,Aarau
Genève-Aéroport,0,860,5390,5191,5734,6265,466,4426,5578,8322,...,769,6949,2433,5311,5051,6420,1184,4555,3385,6396
Genève,8666,0,2558,7849,2047,2747,9167,9998,189,2734,...,4658,1899,7734,1267,1528,3556,3890,8838,5393,8792
Zürich Oerlikon,8433,7513,0,7041,9555,6235,5486,7099,9670,775,...,3152,1585,3943,7555,3073,1021,3843,7989,9692,6873
Zürich HB,5675,161,4297,0,7629,9467,1016,7869,6439,7892,...,7916,8529,878,9268,4887,4859,6331,8571,8684,7208
Thun,5276,2062,64,8006,0,5463,2027,2695,9687,5258,...,6736,391,5892,3561,6184,3099,6278,8392,3104,7215


## Model


Cost = boarding (H/2) + TT + transfer_penalty (penalty + H/2) + crowding_penalty 

#### Start of with all or nothing assignment 

In [9]:
TRANSFER_PENALTY = 600  # seconds

HEADWAY_BY_LINE = {
    'IC1': 600, 'IC2': 1200, 'IC3': 900, 'IC5': 600,
    'IC6': 1800, 'IC8': 1200, 'IC21': 1200,
    'IC51': 1800, 'IC61': 1800, 'IC81': 1200
}

AVG_SPEED = 80_000 / 3600  # m/s ≈ 22.22

We need to track the transfers made 

The most effective approach is to create a multi-layered graph where each node isn't just a station, but a (station, line) pair. This way, transferring between lines at the same station becomes an explicit edge with an associated transfer cost.

In [10]:
def label_correction(t, r, n_node, outflow_link_frst, outflow_link_lst, end_node):
    
    """
    Generate shortest path tree from a given origin node using label correction

    :param t: link cost 
    :param r: origin node index
    :param n_node: number of nodes

    :param outflow_link_frst (outflow_link_lst): list of first (last) outflow link indices for each node
    :param end_node: list of end node indices for each link

    :return u: node label
    :return p: previous link on shortest path for each node
    """

    # initialize
    u = np.inf * np.ones(n_node)
    u[r] = 0
    p = -1 * np.ones(n_node, dtype=int)
    Q = queue.Queue()
    Q.put(r) 

    while Q.qsize() > 0 :

        node_i = Q.get() 

        link_frst = outflow_link_frst[node_i]
        link_lst = outflow_link_lst[node_i]
        
        for link in range(link_frst, link_lst + 1):

            node_j = end_node[link]

            if u[node_j] > u[node_i] + t[link] :

                u[node_j] = u[node_i] + t[link]
                p[node_j] = link
                Q.put(node_j)


    return u, p

In [11]:
n_node = len(df_node)

outflow_link_frst = df_node.frst_out.values
outflow_link_lst = df_node.lst_out.values

start_node = df_link.start_node.values
end_node = df_link.end_node.values

t = df_link['distance [m]'].values


## Mapping string to int for algorthm to work
r = "Lausanne"
node_to_index = {node: idx for idx, node in enumerate(df_node['node_id'])}
r_index = node_to_index[r]
end_node_indices = np.array([node_to_index[n] for n in end_node])

# Now you can call the function with integer indices:
u, p = label_correction(t, r_index, n_node, outflow_link_frst, outflow_link_lst, end_node_indices)

In [12]:
u

array([175887.929, 239968.009, 177474.735, 388148.972,  96888.245,
       104288.077, 333456.236,  65778.35 ,  60226.63 ,  66144.818,
            0.   , 417703.968, 417044.034, 218282.618,  75087.957,
       162514.525, 301307.243, 128152.257, 244168.201, 246310.532,
       217380.834, 222749.251])

In [13]:
p

array([33, 46, 14, 29, 18, 31, 48, 23, 24, 20, -1, 28,  7, 37, 25, 11, 40,
       12, 49, 50, 13, 51])

In [19]:
# Prepare data for Dijkstra algorithm
def prepare_for_dijkstra(network_df):
    
    """Prepare network data for Dijkstra algorithm"""
    
    # Sort by source_id and dest_id
    network_df = network_df.sort_values(['source_id', 'dest_id']).reset_index(drop=True)
    
    # Add link indices
    network_df['link_index'] = network_df.index
    
    # Group by source_id to get first and last outflow links
    grouped = network_df.groupby('source_id')['link_index'].agg(['min', 'max']).reset_index()
    
    # Create node dataframe
    node_df = pd.DataFrame({
        'node_id': grouped['source_id'],
        'frst_out': grouped['min'],
        'lst_out': grouped['max']
    })
    
    # Create mapping from node_id to index
    node_to_idx = {node: idx for idx, node in enumerate(node_df['node_id'])}
    
    # Convert end nodes to indices
    end_node_indices = np.array([node_to_idx.get(node, -1) for node in network_df['dest_id']])
    
    return node_df, node_to_idx, end_node_indices, network_df

In [20]:
# Modified Dijkstra (label correction) algorithm
def dijkstra_with_transfers(df_links, origin_station, origin_line, destination_station):
    """
    Find shortest path with line transfers in a train network
    
    Parameters:
    -----------
    df_links : DataFrame
        Original link dataframe
    origin_station : str
        Starting station
    origin_line : str
        Starting train line
    destination_station : str
        Destination station (can arrive on any line)
    
    Returns:
    --------
    path : list
        List of (station, line) pairs on shortest path
    cost : float
        Total travel time including transfers
    """
    
    # 1. Create full network with transfers
    bidirectional_network = create_bidirectional_network(df_links)
    full_network = add_transfer_edges(bidirectional_network)
    full_network = calculate_travel_times(full_network)
    full_network = create_node_line_pairs(full_network)
    
    # 2. Prepare data structures for Dijkstra
    node_df, node_to_idx, end_node_indices, network_df = prepare_for_dijkstra(full_network)
    
    # 3. Initialize algorithm variables
    n_node = len(node_df)
    outflow_link_frst = node_df['frst_out'].values
    outflow_link_lst = node_df['lst_out'].values
    link_costs = network_df['travel_time'].values
    
    # 4. Find index of origin node
    origin_node_id = f"{origin_station}_{origin_line}"
    if origin_node_id not in node_to_idx:
        return None, float('inf')  # Origin station-line pair not found
    origin_idx = node_to_idx[origin_node_id]
    
    # 5. Run label correction algorithm
    labels, prev_links = label_correction(
        link_costs, origin_idx, n_node, 
        outflow_link_frst, outflow_link_lst, end_node_indices
    )
    
    # 6. Find best path to destination (any line)
    destination_nodes = [node for node in node_to_idx.keys() if node.startswith(f"{destination_station}_")]
    best_dest_idx = None
    best_cost = float('inf')
    
    for dest_node in destination_nodes:
        dest_idx = node_to_idx[dest_node]
        if labels[dest_idx] < best_cost:
            best_cost = labels[dest_idx]
            best_dest_idx = dest_idx
    
    if best_dest_idx is None:
        return None, float('inf')  # No path found
    
    # 7. Reconstruct path
    path = []
    current = best_dest_idx
    
    while current != origin_idx:
        node_id = node_df['node_id'][current]
        path.append(node_id)
        
        prev_link = prev_links[current]
        if prev_link == -1:
            return None, float('inf')  # No path found
            
        current = node_to_idx[network_df['source_id'][prev_link]]
    
    path.append(node_df['node_id'][origin_idx])
    path.reverse()
    
    return path, best_cost

In [21]:
# Example usage
origin_station = "Lausanne"
origin_line = "IC1"
destination_station = "Olten"

path, travel_time = dijkstra_with_transfers(df_links, origin_station, origin_line, destination_station)

if path is not None:
    print(f"Total travel time: {travel_time:.2f} minutes")
    print("Path:")
    for i, node in enumerate(path):
        station, line = node.split('_')
        print(f"{i+1}. {station} (Line {line})")
        
        # Show transfers
        if i < len(path) - 1:
            next_station, next_line = path[i+1].split('_')
            if station == next_station and line != next_line:
                print(f"   Transfer to Line {next_line}")
else:
    print("No path found.")

Total travel time: 126.89 minutes
Path:
1. Lausanne (Line IC1)
2. Fribourg/Freiburg (Line IC1)
3. Bern (Line IC1)
   Transfer to Line IC6
4. Bern (Line IC6)
5. Olten (Line IC6)


In [22]:
path

['Lausanne_IC1', 'Fribourg/Freiburg_IC1', 'Bern_IC1', 'Bern_IC6', 'Olten_IC6']